In [ ]:
# Load a sample recommendation dataset and explore it to understand user-item interactions.

import pandas as pd

# Example user-item interaction data
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 4, 4],
    'item_id': [101, 102, 103, 101, 104, 102, 104, 105, 101, 105],
    'rating': [5, 4, 3, 4, 5, 2, 5, 3, 4, 5]
}
df = pd.DataFrame(data)

# Display dataset
print("User-Item Interaction Data:\n", df)

In [ ]:
# Use matrix factorization to implement collaborative filtering, finding patterns in user-item interactions.

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Create a user-item matrix
user_item_matrix = df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Apply TruncatedSVD for dimensionality reduction
svd = TruncatedSVD(n_components=2, random_state=42)
latent_matrix = svd.fit_transform(user_item_matrix)

# Calculate similarity between users
user_similarity = cosine_similarity(latent_matrix)
print("User Similarity Matrix:\n", user_similarity)

In [ ]:
# Recommend items to a user based on the items liked by similar users using collaborative filtering.

import numpy as np

# Find the most similar user to user 1
target_user = 1
similar_user_idx = np.argsort(-user_similarity[target_user - 1])[1]  # Ignore self-similarity

# Get items liked by the similar user but not yet rated by the target user
similar_user_items = user_item_matrix.iloc[similar_user_idx]
target_user_items = user_item_matrix.iloc[target_user - 1]

# Recommend items with high ratings from the similar user that the target user hasn't rated
recommendations = similar_user_items[target_user_items == 0].sort_values(ascending=False)
print("Recommended Items for User 1:\n", recommendations)

In [ ]:
# Implement content-based filtering to recommend items similar to those the user has previously liked.

# Example item content (features describing items)
item_features = pd.DataFrame({
    'item_id': [101, 102, 103, 104, 105],
    'genre': ['Action', 'Adventure', 'Action', 'Comedy', 'Adventure'],
    'length': [120, 150, 110, 90, 130]
})

# Merge user-item ratings with item features
df_content = df.merge(item_features, on='item_id')
user_preferences = df_content[df_content['user_id'] == target_user].groupby('genre').mean()['rating']

# Recommend items based on similar genre preference
recommend_content = item_features[item_features['genre'].isin(user_preferences.index)]
print("Content-Based Recommendations:\n", recommend_content)

In [ ]:
# Combine collaborative and content-based recommendations for a hybrid approach.

# Get top items from collaborative filtering
top_collab_items = recommendations.index[:2].tolist()

# Get top items from content-based filtering
top_content_items = recommend_content['item_id'].tolist()

# Combine recommendations, removing duplicates
hybrid_recommendations = list(set(top_collab_items + top_content_items))
print("Hybrid Recommendations for User 1:\n", hybrid_recommendations)